In [38]:
y_m <- 8 # minimum data value
alpha <- 0.5 # shape parameter between 0 and 1
n <- 20 # sample size

qg025 <- qgamma(0.025,n, n)
qg975 <- qgamma(0.975,n , n )

# qt025 <- qt(0.025, n-1)
qt975 <- qt(0.975, n-1)

# Run experiments

In [39]:
T = 10000 # number of trials to run

alpha_hatMLE <- numeric(T)
alpha_hatMOM <- numeric(T)
exact.lower <- numeric(T) 
exact.upper <- numeric(T)
CLT.lower <- numeric(T)
CLT.upper <- numeric(T)
for(k in 1:T){
    X <- rexp(n, rate = alpha) # Exp(alpha) random variables
    Y <- y_m * exp(X) # Pareto(y_m, alpha) random variables

    alpha_hatMLE[k] <- 1/ mean(log(Y / y_m)) # MLE formula
    alpha_hatMOM[k] <- mean(Y)/(mean(Y)+y_m)

    exact.lower[k] <- alpha_hatMLE[k] * qg025
    exact.upper[k] <- alpha_hatMLE[k] * qg975 # based on exact MLE CI (part d)

    CLT.lower[k] <- 1/(1/alpha_hatMLE[k] + qt975 * sd(X) / sqrt(n))# based on t distribution CI (part e)
    CLT.upper[k] <- 1/(1/alpha_hatMLE[k] - qt975 * sd(X) / sqrt(n))# based on t distribution CI (part e)

    
}


In [40]:
rmse_MLE <- sqrt(mean((alpha_hatMLE - alpha)^2))
rmse_MOM <- sqrt(mean((alpha_hatMOM - alpha)^2))

In [41]:
coverage_exact <- mean((alpha >= exact.lower) & (alpha <= exact.upper)) # compute for each CI
coverage_CLT <- mean((alpha >= CLT.lower) & (alpha <= CLT.upper)) # compute for each CI
# coverage <- mean((alpha >= exact.lower) & (alpha <= exact.upper)) # compute for each CI

In [43]:
coverage_exact

[1] 0.949

In [42]:
coverage_CLT

[1] 0.9175

In [ ]:


# set.seed(123)
# B <- 1e4
# boot_t_alphaMLE <- numeric(B)
# boot_t_alphaMOM <- numeric(B)

# for(b in 1:B){
#     Y_boot <- sample(Y, replace=TRUE)

#     boot_alphaMLE <- 1/ mean(log(Y_boot / y_m))
#     boot_alphaMOM <- mean(Y_boot)/(mean(Y_boot)+y_m)

#     boot_t_alphaMLE[b] <- (boot_alphaMLE - alpha_hatMLE[k])


# }


# alpha_hatMLE[k] - quantile(bootMLE,c(0.975, 0.025))

# mean(diff) - quantile(boot_t,c(0.975, 0.025)) * sd(diff) /sqrt(n)



# bootMLE.lower[k] <- # based on the bootstrap percentile method applied to the MLE
# bootMLE.upper[k] <- # based on the bootstrap percentile method applied to the MLE
# bootMOM.lower[k] <- # based on the bootstrap percentile method applied to the MOM estimate
# bootMOM.upper[k] <- # based on the bootstrap percentile method applied to the MOM estimatealpha_hatMLE